### HW2: Task 2 - Dialogue act detection

#### Khandokar Md. Nayem (knayem) 

#### Approach:
We try to classify text using RNNs to identify the goal or Dialogue Act in a conversation. As dataset, we use the Switchboard Dialog Act Corpus. We generate a sub corpus from the Switchboard Dialog Act Corpus extracting only the sentences which represent `greeting`, `goodby` and `order`. We saved the sub corpus in a file named "swad_small.txt". This text file is attached.  

We preprocess each sentence by removing punctuations and lowering each words. Then we take the stem of each words and create a tokenize list for each sentence. We create word embedding for each word using Word2Vec with N-gram language model. It gives us a vector of length 100 for each word.  

We consider 80% of the dataset as train dataset and 20% as test dataset from the whole dataset. And one-hot encodeing as output label.

We use keras sequential model for this task. To classify the dialogues, we experimented with two different model. First one with is a single LSTM with pre-trained word embeddings. We use 0.0 drop out and last pooling for our network. Then we add a densed output layer where we used sigmoid as a activation function as we are trying to classify the dialogue acts. To calculate the loss function we used cross-entropy and AdamOptimizer as optimizer function.

Second one is a single GRU (Gated Recurrent Unit) with pre-trained word embeddings. Like Hamed et. al we use 0.2 drop out and last pooling for our network. Then we add a densed output layer where we used sigmoid, cross-entropy as loss function and AdamOptimizer as optimizer.

While training the model, we split the train dataset into 5% as validation set. We consider 100 epoch and 64 batch size. 

#### Accuracy:
After evaluating the model we found the accuracy of: 78.51% in both cases. This is almost same as the paper.

#### Ideas for improvement:
Here we used the small dataset of Swda and we considered only 3 dialogue acts. So our dataset is pretty small. Larger size dataset can significanltly improve the result. Also we only use N-gram model as vectorization scheme. Other linguistic feature like dependency tree, lemma can be useful to improve the accuracy. So in future, we will incorporate these features in the vectorization scheme and investigate those.

In [19]:
from __future__ import print_function

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

from keras.models import Model
from keras.layers import Input, LSTM, GRU, Dense, Dropout, Bidirectional
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.np_utils import to_categorical

import numpy as np

import csv

import gensim
import nltk
from nltk.stem.lancaster import LancasterStemmer
import string

### File defined

In [2]:
stemmer = LancasterStemmer()
ignore_words = []

# Set the directory you want to start from
rootDir = './swda'

file_name_sm = './swda_small.txt'

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.33
set_session(tf.Session(config=config))

### Create Train File for 3 classes 

In [3]:
if os.path.exists(file_name_sm):
    mode = 'w' # append if already exists
else:
    mode = 'a' # make a new file if not

with open(file_name_sm, mode) as train_file:
    mode = 'a' # make a new file if not
    
    for dirName, subdirList, fileList in os.walk(rootDir):
        print('Found directory:', dirName)
        for fname in fileList:
#             print(fname)
            f=os.path.join(dirName,fname)

            with open(f, 'r') as csvfile:
    #             spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
                spamreader = csv.reader(csvfile)
                for row in spamreader:
#                     print(row)
                    # greeting
                    if row[4]=='qo' or row[4]=='fp':
                        train_file.write(' '.join(['greeting',row[8],'\n'] ))
                    
                    # goodbye
                    elif row[4]=='fc':
                        train_file.write(' '.join(['goodbye',row[8],'\n'] ))
                                         
                    # request
                    elif row[4]=='ad':
                        train_file.write(' '.join(['request',row[8],'\n'] ))
    

Found directory: ./swda
Found directory: ./swda/sw13utt
Found directory: ./swda/sw06utt
Found directory: ./swda/sw11utt
Found directory: ./swda/sw01utt
Found directory: ./swda/sw03utt
Found directory: ./swda/sw00utt
Found directory: ./swda/sw02utt
Found directory: ./swda/sw07utt
Found directory: ./swda/sw12utt
Found directory: ./swda/sw10utt
Found directory: ./swda/sw08utt
Found directory: ./swda/sw09utt
Found directory: ./swda/sw04utt
Found directory: ./swda/sw05utt


### Word2Vec Model and Tokenize

In [7]:
def preProcessing(line):
    line = line.split()[1:]
    line = [''.join(c for c in s if c not in string.punctuation) for s in line]
    line = [s.lower() for s in line if s]
    line = [s for s in line if len(s)>1 or s=='i' or s=='a']
    line = [stemmer.stem(w) for w in line if w not in ignore_words]
    return line

class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for line in open(file_name_sm, 'r'):
#             print(preProcessing(line))
            yield preProcessing(line)
 
sentences = MySentences(file_name_sm) # a memory-friendly iterator
model = gensim.models.Word2Vec(sentences, min_count=1)

### Train and Test data prepare

In [8]:
Classes = np.array(['greeting','goodbye','request'])
Max_RNN = 100

train_set={'x':[],'y':[]}
dev_set={'x':[],'y':[]}
test_set={'x':[],'y':[]}

for line in open(file_name_sm, 'r'):
    tag = line.split()[0]
    
    ws = preProcessing(line)
    for e in np.arange(Max_RNN):
        if e < len(ws):
            w = ws[e]
#             print( len([model.wv[w][:5]]) ,'w: ',[model.wv[w][:5]])
            vec = np.array([model.wv[w]],ndmin=2) if e==0 else np.append(vec, [model.wv[w]],axis=0)
#             print(vec.shape,'V: ',vec)
        else:
#             print(np.zeros((1,5)).shape,'w: ',np.zeros((1,5)))
            vec = np.zeros((1,100)) if e==0 else np.append(vec, np.zeros((1,100)),axis=0)
#             print(vec.shape,'V: ',vec)
            
        
#     print('U-', vec.shape)
    
    DataType = np.random.choice(2, 1, p=[0.8, 0.2])
    if DataType == 0:
        if len(train_set['x'])<1:
            train_set['x'] = np.array([vec],ndmin=2)
    #         train_set['y'] = np.array( [np.array([1*(Classes == line[0])], ndmin=2).T] )
            train_set['y'] = np.array(  np.where(Classes==tag) ,ndmin=2)
#             print(train_set['y'],tag)
        else:
            train_set['x'] = np.append(train_set['x'], [vec], axis=0)
    #         train_set['y'] = np.append(train_set['y'], [np.array([1*(Classes == line[0])], ndmin=2).T],  axis=0 )
            train_set['y'] = np.append(train_set['y'],  np.where(Classes==tag),  axis=0 )
#             print(train_set['y'],tag)
            
    elif DataType == 1:
        if len(test_set['x'])<1:
            test_set['x'] = np.array([vec],ndmin=2)
            test_set['y'] = np.array(  np.where(Classes==tag) ,ndmin=2)
        else:
            test_set['x'] = np.append(test_set['x'], [vec], axis=0)
            test_set['y'] = np.append(test_set['y'],  np.where(Classes==tag),  axis=0 )
            
#     print('X-',train_set['x'].shape, train_set['y'].shape)
    
    
# Convert labels to categorical one-hot encoding
# one_hot_labels = keras.utils.to_categorical(train_set['y'], num_classes=3)
train_set['y'] = to_categorical(train_set['y'], num_classes=Classes.size)
test_set['y'] = to_categorical(test_set['y'], num_classes=Classes.size)
print('Q-', train_set['x'].shape )
print('Q-', train_set['y'].shape )
print('Q-', test_set['x'].shape )
print('Q-', test_set['y'].shape )

Q- (3144, 100, 100)
Q- (3144, 3)
Q- (777, 100, 100)
Q- (777, 3)


### LSTM Model

In [17]:
# create the model

model = Sequential()

model.add(LSTM(Max_RNN, return_sequences=False, input_shape=(Max_RNN,100)))

model.add(Dense(Classes.size, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(train_set['x'], train_set['y'],validation_split=0.05, shuffle=True, nb_epoch=100, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(test_set['x'], test_set['y'], verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 303       
Total params: 80,703
Trainable params: 80,703
Non-trainable params: 0
_________________________________________________________________
None


/N/u/knayem/.conda/envs/b659/lib/python3.6/site-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 2986 samples, validate on 158 samples
Epoch 1/100
2986/2986 [==============================] - 8s 3ms/step - loss: 0.9292 - acc: 0.6025 - val_loss: 0.7649 - val_acc: 0.6899
Epoch 2/100
2986/2986 [==============================] - 6s 2ms/step - loss: 0.8461 - acc: 0.6072 - val_loss: 0.7874 - val_acc: 0.6899
Epoch 3/100
2986/2986 [==============================] - 6s 2ms/step - loss: 0.8403 - acc: 0.6075 - val_loss: 0.7633 - val_acc: 0.6899
Epoch 4/100
2986/2986 [==============================] - 6s 2ms/step - loss: 0.8430 - acc: 0.6072 - val_loss: 0.7614 - val_acc: 0.6899
Epoch 5/100
2986/2986 [==============================] - 6s 2ms/step - loss: 0.8374 - acc: 0.6092 - val_loss: 0.7597 - val_acc: 0.6899
Epoch 6/100
2986/2986 [==============================] - 6s 2ms/step - loss: 0.8322 - acc: 0.6082 - val_loss: 0.7559 - val_acc: 0.6899
Epoch 7/100
2986/2986 [==============================] - 6s 2ms/step - loss: 0.8284 - acc: 0.6139 - val_loss: 0.7413 - val_acc: 0.6899
Epoch 8/

Epoch 61/100
2986/2986 [==============================] - 6s 2ms/step - loss: 0.6230 - acc: 0.7384 - val_loss: 0.5331 - val_acc: 0.7848
Epoch 62/100
2986/2986 [==============================] - 7s 2ms/step - loss: 0.6018 - acc: 0.7488 - val_loss: 0.5475 - val_acc: 0.7468
Epoch 63/100
2986/2986 [==============================] - 7s 2ms/step - loss: 0.5870 - acc: 0.7498 - val_loss: 0.5876 - val_acc: 0.8165
Epoch 64/100
2986/2986 [==============================] - 7s 2ms/step - loss: 0.5821 - acc: 0.7612 - val_loss: 0.5163 - val_acc: 0.7848
Epoch 65/100
2986/2986 [==============================] - 7s 2ms/step - loss: 0.6146 - acc: 0.7605 - val_loss: 0.5108 - val_acc: 0.7975
Epoch 66/100
2986/2986 [==============================] - 6s 2ms/step - loss: 0.5891 - acc: 0.7582 - val_loss: 0.5165 - val_acc: 0.7848
Epoch 67/100
2986/2986 [==============================] - 6s 2ms/step - loss: 0.5731 - acc: 0.7629 - val_loss: 0.5367 - val_acc: 0.7595
Epoch 68/100
2986/2986 [========================

### GRU Model

In [22]:
# create the model

model = Sequential()
model.add(Bidirectional(GRU(Max_RNN, return_sequences=False), input_shape=(Max_RNN,100)))
model.add(Dropout(0.2))
model.add(Dense(Classes.size, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(train_set['x'], train_set['y'],validation_split=0.05, shuffle=True, nb_epoch=100, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(test_set['x'], test_set['y'], verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_3 (Bidirection (None, 200)               120600    
_________________________________________________________________
dropout_7 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 603       
Total params: 121,203
Trainable params: 121,203
Non-trainable params: 0
_________________________________________________________________
None


/N/u/knayem/.conda/envs/b659/lib/python3.6/site-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 2986 samples, validate on 158 samples
Epoch 1/100
2986/2986 [==============================] - 12s 4ms/step - loss: 0.9727 - acc: 0.5908 - val_loss: 0.8135 - val_acc: 0.6899
Epoch 2/100
2986/2986 [==============================] - 10s 3ms/step - loss: 0.8429 - acc: 0.6068 - val_loss: 0.7819 - val_acc: 0.6899
Epoch 3/100
2986/2986 [==============================] - 10s 3ms/step - loss: 0.8391 - acc: 0.6082 - val_loss: 0.7811 - val_acc: 0.6899
Epoch 4/100
2986/2986 [==============================] - 10s 3ms/step - loss: 0.8352 - acc: 0.6065 - val_loss: 0.7635 - val_acc: 0.6899
Epoch 5/100
2986/2986 [==============================] - 10s 3ms/step - loss: 0.8229 - acc: 0.6065 - val_loss: 0.7592 - val_acc: 0.6899
Epoch 6/100
2986/2986 [==============================] - 10s 3ms/step - loss: 0.8141 - acc: 0.6065 - val_loss: 0.7433 - val_acc: 0.6899
Epoch 7/100
2986/2986 [==============================] - 10s 3ms/step - loss: 0.7950 - acc: 0.6065 - val_loss: 0.7179 - val_acc: 0.6899
E

Epoch 61/100
2986/2986 [==============================] - 9s 3ms/step - loss: 0.3054 - acc: 0.8881 - val_loss: 0.6892 - val_acc: 0.7785
Epoch 62/100
2986/2986 [==============================] - 9s 3ms/step - loss: 0.3091 - acc: 0.8838 - val_loss: 0.6671 - val_acc: 0.7722
Epoch 63/100
2986/2986 [==============================] - 9s 3ms/step - loss: 0.2980 - acc: 0.8932 - val_loss: 0.6819 - val_acc: 0.7722
Epoch 64/100
2986/2986 [==============================] - 10s 3ms/step - loss: 0.2907 - acc: 0.8938 - val_loss: 0.6939 - val_acc: 0.7658
Epoch 65/100
2986/2986 [==============================] - 10s 3ms/step - loss: 0.2999 - acc: 0.8875 - val_loss: 0.6770 - val_acc: 0.7658
Epoch 66/100
2986/2986 [==============================] - 10s 3ms/step - loss: 0.2887 - acc: 0.8952 - val_loss: 0.6862 - val_acc: 0.7975
Epoch 67/100
2986/2986 [==============================] - 10s 3ms/step - loss: 0.2811 - acc: 0.9019 - val_loss: 0.6753 - val_acc: 0.7722
Epoch 68/100
2986/2986 [====================